In [1]:
import pandas as pd
import numpy as np

# === Cargar ===
df = pd.read_csv("cars_clean.csv", low_memory=False)

# === Normalizar texto para evitar duplicados por mayúsculas/espacios ===
for c in ["manufacturer","model","fuel","transmission","state","region"]:
    if c in df.columns:
        df[c] = (df[c].astype(str).str.strip().str.lower()
                           .str.replace(r"\s+"," ", regex=True)
                           .replace({"nan": np.nan}))

# === Asegurar numéricos ===
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["odometer"] = pd.to_numeric(df["odometer"], errors="coerce")

# === Filtro de años de análisis ===
df = df[(df["year"] >= 2011) & (df["year"] <= 2021)].copy()

# === Cortes suaves para no contaminar (outliers extremos) ===
df["price"] = df["price"].clip(lower=500, upper=200_000)
df["odometer"] = df["odometer"].clip(lower=0, upper=500_000)

# === Quitar duplicados exactos en columnas clave (evita filas repetidas) ===
dedup_cols = ["manufacturer","model","year","price","odometer","state","region"]
df = df.drop_duplicates(subset=[c for c in dedup_cols if c in df.columns]).reset_index(drop=True)

print("Filas finales (2011–2021):", len(df))
df.head(3)



Filas finales (2011–2021): 218268


,price,year,manufacturer,model,odometer,fuel,transmission,state,region,car_age,ref_price_model,ref_price_brand,ref_price,depreciation_pct,anomaly,cluster,cluster_label
0,33590,2014.0,gmc,sierra 1500 crew cab slt,57923.0,gas,other,al,auburn,11.0,NaN,NaN,13950.0,-10,1.0,0.0,newer-mid/high
1,39590,2020.0,chevrolet,silverado 1500 crew,19160.0,gas,other,al,auburn,5.0,NaN,NaN,13950.0,-10,1.0,0.0,newer-mid/high
2,30990,2017.0,toyota,tundra double cab sr,41124.0,gas,other,al,auburn,8.0,NaN,NaN,13950.0,-10,1.0,0.0,newer-mid/high


In [2]:
from IPython.display import display

# ========== Top 10 marcas ==========
top_brands = df["manufacturer"].value_counts().head(10).rename("count")
display(top_brands)

# Lista de esas marcas (para usarla después)
top10_list = top_brands.index.tolist()

# ========== Top 10 modelos (global, sin repetir marca+modelo) ==========
top_models = (df.groupby("model")
                .size()
                .rename("count")
                .sort_values(ascending=False)
                .head(10)
                .reset_index())
display(top_models)

# ========== Publicaciones por año (2011–2021) ==========
year_counts = (df.groupby("year")
                 .size()
                 .rename("count")
                 .reindex(range(2011, 2022), fill_value=0)
                 .reset_index())
display(year_counts)

# ========== Total de anuncios 2011–2021 ==========
total_ads_2011_2021 = int(year_counts["count"].sum())
print("Total de anuncios (2011–2021):", total_ads_2011_2021)

# ========== Odómetro promedio (solo Top 10 marcas) ==========
avg_odo_top_brands = (df[df["manufacturer"].isin(top10_list)]
                        .groupby("manufacturer")["odometer"]
                        .mean()
                        .round(0)
                        .sort_values(ascending=False))
display(avg_odo_top_brands)

# (Opcional) Guardar CSVs para tu dashboard (Looker/Flourish)
brand_year = (df.groupby(["year","manufacturer"]).size()
                .reset_index(name="ads_count"))
brand_year.to_csv("brand_year.csv", index=False)

top_brands.to_csv("brand_totals.csv")            # marca, count
year_counts.to_csv("year_totals.csv", index=False)
avg_odo_top_brands.to_csv("avg_odometer_top_brands.csv")
top_models.to_csv("top_models.csv", index=False)
from IPython.display import display

# ========== Top 10 marcas ==========
top_brands = df["manufacturer"].value_counts().head(10).rename("count")
display(top_brands)

# Lista de esas marcas (para usarla después)
top10_list = top_brands.index.tolist()

# ========== Top 10 modelos (global, sin repetir marca+modelo) ==========
top_models = (df.groupby("model")
                .size()
                .rename("count")
                .sort_values(ascending=False)
                .head(10)
                .reset_index())
display(top_models)

# ========== Publicaciones por año (2011–2021) ==========
year_counts = (df.groupby("year")
                 .size()
                 .rename("count")
                 .reindex(range(2011, 2022), fill_value=0)
                 .reset_index())
display(year_counts)

# ========== Total de anuncios 2011–2021 ==========
total_ads_2011_2021 = int(year_counts["count"].sum())
print("Total de anuncios (2011–2021):", total_ads_2011_2021)

# ========== Odómetro promedio (solo Top 10 marcas) ==========
avg_odo_top_brands = (df[df["manufacturer"].isin(top10_list)]
                        .groupby("manufacturer")["odometer"]
                        .mean()
                        .round(0)
                        .sort_values(ascending=False))
display(avg_odo_top_brands)

# (Opcional) Guardar CSVs para tu dashboard (Looker/Flourish)
brand_year = (df.groupby(["year","manufacturer"]).size()
                .reset_index(name="ads_count"))
brand_year.to_csv("brand_year.csv", index=False)

top_brands.to_csv("brand_totals.csv")            # marca, count
year_counts.to_csv("year_totals.csv", index=False)
avg_odo_top_brands.to_csv("avg_odometer_top_brands.csv")
top_models.to_csv("top_models.csv", index=False)


manufacturer
ford         35814
chevrolet    26649
toyota       15500
ram          10604
nissan       10311
jeep          9861
gmc           9161
honda         9111
bmw           8931
dodge         6795
Name: count, dtype: int64

,model,count
0,f-150,3872
1,silverado 1500,2515
2,1500,2184
3,escape,1463
4,altima,1349
5,2500,1310
6,explorer,1299
7,equinox,1192
8,grand cherokee,1182
9,silverado,1137


,year,count
0,2011,15843
1,2012,18270
2,2013,23233
3,2014,22159
4,2015,22917
5,2016,22656
6,2017,27194
7,2018,28444
8,2019,20152
9,2020,16067


Total de anuncios (2011–2021): 218268


manufacturer
ford         81235.0
ram          75892.0
chevrolet    74354.0
gmc          74021.0
honda        73266.0
nissan       71702.0
dodge        69797.0
toyota       68170.0
jeep         62203.0
bmw          59557.0
Name: odometer, dtype: float64

manufacturer
ford         35814
chevrolet    26649
toyota       15500
ram          10604
nissan       10311
jeep          9861
gmc           9161
honda         9111
bmw           8931
dodge         6795
Name: count, dtype: int64

,model,count
0,f-150,3872
1,silverado 1500,2515
2,1500,2184
3,escape,1463
4,altima,1349
5,2500,1310
6,explorer,1299
7,equinox,1192
8,grand cherokee,1182
9,silverado,1137


,year,count
0,2011,15843
1,2012,18270
2,2013,23233
3,2014,22159
4,2015,22917
5,2016,22656
6,2017,27194
7,2018,28444
8,2019,20152
9,2020,16067


Total de anuncios (2011–2021): 218268


manufacturer
ford         81235.0
ram          75892.0
chevrolet    74354.0
gmc          74021.0
honda        73266.0
nissan       71702.0
dodge        69797.0
toyota       68170.0
jeep         62203.0
bmw          59557.0
Name: odometer, dtype: float64

In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# === Imputación con medianas para no perder filas ===
price_med = df["price"].median(skipna=True)
odo_med   = df["odometer"].median(skipna=True)
df["_price_imp"] = df["price"].fillna(price_med)
df["_odo_imp"]   = df["odometer"].fillna(odo_med)

# === Escalado + KMeans ===
X = df[["_price_imp","_odo_imp"]].to_numpy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(X_scaled)

# === Etiquetas amigables ordenadas por precio promedio del cluster ===
order = df.groupby("cluster")["_price_imp"].mean().sort_values().index.tolist()
name_map = {order[0]:"económico/alto km", order[1]:"medio", order[2]:"premium/bajo km"}
df["cluster_name"] = df["cluster"].map(name_map)

# === Tabla resumen por cluster ===
cluster_summary = (df.groupby(["cluster","cluster_name"])
                     .agg(cantidad_autos=("cluster","size"),
                          precio_promedio=("price","mean"),
                          odometro_promedio=("odometer","mean"))
                     .reset_index()
                     .sort_values("cluster"))

cluster_summary["precio_promedio"] = cluster_summary["precio_promedio"].round(2)
cluster_summary["odometro_promedio"] = cluster_summary["odometro_promedio"].round(2)

total = int(cluster_summary["cantidad_autos"].sum())
cluster_summary["porcentaje"] = (cluster_summary["cantidad_autos"]/total*100).round(2)

# Fila Total
fila_total = pd.DataFrame({
    "cluster": ["Total"],
    "cluster_name": ["Total"],
    "cantidad_autos": [total],
    "precio_promedio": [np.nan],
    "odometro_promedio": [np.nan],
    "porcentaje": [100.00]
})

tabla_clusters = pd.concat([cluster_summary, fila_total], ignore_index=True)
display(tabla_clusters)

# (Opcional) Guardar
tabla_clusters.to_csv("cluster_summary_2011_2021.csv", index=False)

# Chequeo coherencia
print("Chequeo — Total por años:", total_ads_2011_2021, "| Total en clusters:", total)


,cluster,cluster_name,cantidad_autos,precio_promedio,odometro_promedio,porcentaje
0,0,medio,95154,22225.48,40891.08,43.60
1,1,económico/alto km,74897,14703.20,125146.97,34.31
2,2,premium/bajo km,48217,44749.01,33637.33,22.09
3,Total,Total,218268,NaN,NaN,100.00


Chequeo — Total por años: 218268 | Total en clusters: 218268


In [ ]:
# Asegúrate de tener en memoria: df, top_brands, top_models, year_counts, avg_odo_top_brands, tabla_clusters
top_brands.rename_axis("manufacturer").reset_index().to_csv("brand_totals.csv", index=False)
top_models.to_csv("top_models.csv", index=False)
year_counts.to_csv("year_totals.csv", index=False)
avg_odo_top_brands.rename("avg_odometer").rename_axis("manufacturer").reset_index().to_csv("avg_odometer_top_brands.csv", index=False)

# marcas x año (para barras por año o filtro de año)
brand_year = df.groupby(["year","manufacturer"]).size().reset_index(name="ads_count")
brand_year.to_csv("brand_year.csv", index=False)

# clusters
tabla_clusters.to_csv("cluster_summary_2011_2021.csv", index=False)
print("CSV listos ")


CSV listos ✅
